# Backtest Analysis

This notebook runs a comprehensive backtest of the AI-enhanced 60/40 portfolio strategy and compares it against traditional benchmarks.

## Objectives:
1. Generate AI-driven portfolio allocations
2. Backtest the strategy
3. Compare against benchmarks (Buy & Hold SPY, Traditional 60/40)
4. Analyze performance metrics
5. Visualize results

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from data_acquisition import DataAcquisition
from feature_engineering import FeatureEngineer
from ml_model import PortfolioMLModel
from backtester import PortfolioBacktester

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("Libraries imported successfully!")

## 1. Load Data and Models

In [ ]:
# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Initialize components
data_acq = DataAcquisition(config)
feature_eng = FeatureEngineer(config)
ml_model = PortfolioMLModel(config)
backtester = PortfolioBacktester(config)

print("Components initialized successfully!")

In [ ]:
# Fetch data
prices, returns, indicators = data_acq.get_full_dataset()

print(f"Data loaded: {len(prices)} periods")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")

## 2. Engineer Features and Train Models

In [ ]:
# Engineer features
features_raw = feature_eng.engineer_all_features(indicators)
features = feature_eng.prepare_features_for_training(features_raw)

print(f"Features prepared: {features.shape}")

In [ ]:
# Train models
print("Training models...")
ml_model.train_all_models(X_train, y_train)
print("Models trained successfully!")


## 3. Generate Portfolio Allocations

In [ ]:
# Predict returns for entire dataset
predicted_returns = ml_model.predict_returns(features)

print(f"Predicted returns shape: {predicted_returns.shape}")
print("\nSample predictions:")
display(predicted_returns.head())

In [ ]:
# Calculate optimal allocations
allocations = ml_model.calculate_optimal_allocations(predicted_returns)

print(f"Allocations shape: {allocations.shape}")
print("\nSample allocations:")
display(allocations.head(10))

In [ ]:
# Allocation statistics
print("Allocation statistics:")
display(allocations.describe())

print("\nAverage allocations:")
avg_allocations = allocations.mean()
for asset, alloc in avg_allocations.items():
    print(f"  {asset}: {alloc:.2%}")

In [ ]:
# Visualize allocations over time
fig, ax = plt.subplots(figsize=(14, 7))

ax.stackplot(allocations.index, allocations.T, labels=allocations.columns, alpha=0.8)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Allocation', fontsize=12)
ax.set_title('AI Portfolio Allocations Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=10, bbox_to_anchor=(1.02, 1))
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Run Backtests

In [ ]:
# Backtest AI strategy
print("Backtesting AI Portfolio...")
ai_results = backtester.backtest_strategy(allocations, returns, prices)

# Backtest benchmark (Buy & Hold SPY)
print("Backtesting Buy & Hold SPY...")
benchmark_results = backtester.create_benchmark_strategy(returns, prices, 'SPY')

# Backtest traditional 60/40
print("Backtesting Traditional 60/40...")
traditional_6040_results = backtester.create_traditional_6040(returns, prices)

print("\nAll backtests completed!")

In [ ]:
# Display sample results
print("AI Portfolio Results (first 10 periods):")
display(ai_results.head(10))

print("\nAI Portfolio Results (last 10 periods):")
display(ai_results.tail(10))

## 5. Calculate Performance Metrics

In [ ]:
# Calculate metrics for all strategies
strategies = {
    'AI Portfolio': ai_results,
    'Buy & Hold SPY': benchmark_results,
    'Traditional 60/40': traditional_6040_results
}

comparison = backtester.compare_strategies(strategies)

print("="*70)
print("PERFORMANCE COMPARISON")
print("="*70)
display(comparison)

In [ ]:
# Highlight key metrics
key_metrics = ['Sharpe Ratio', 'Sortino Ratio', 'CAGR', 'Max Drawdown', 
               'Volatility', 'Calmar Ratio']

key_comparison = comparison[key_metrics]

fig, ax = plt.subplots(figsize=(12, 6))
key_comparison.T.plot(kind='bar', ax=ax, width=0.8, edgecolor='black')
ax.set_ylabel('Value', fontsize=12)
ax.set_title('Key Performance Metrics Comparison', fontsize=14, fontweight='bold')
ax.legend(fontsize=10, loc='best')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate improvements over benchmark
print("\n" + "="*70)
print("AI PORTFOLIO IMPROVEMENTS OVER BENCHMARKS")
print("="*70)

for benchmark_name in ['Buy & Hold SPY', 'Traditional 60/40']:
    print(f"\nvs {benchmark_name}:")
    
    for metric in key_metrics:
        ai_value = comparison.loc['AI Portfolio', metric]
        bench_value = comparison.loc[benchmark_name, metric]
        
        if metric == 'Max Drawdown':
            # Lower is better for drawdown
            improvement = ((bench_value - ai_value) / bench_value) * 100
        else:
            # Higher is better for other metrics
            improvement = ((ai_value - bench_value) / bench_value) * 100
        
        print(f"  {metric}: {improvement:+.2f}%")

## 6. Portfolio Value Analysis

In [ ]:
# Plot portfolio values
fig, ax = plt.subplots(figsize=(14, 7))

for name, results in strategies.items():
    ax.plot(results.index, results['Portfolio_Value'], label=name, linewidth=2.5)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Portfolio Value ($)', fontsize=12)
ax.set_title('Portfolio Value Over Time', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
plt.tight_layout()
plt.show()

# Print final values
print("\nFinal Portfolio Values:")
for name, results in strategies.items():
    final_value = results['Portfolio_Value'].iloc[-1]
    total_return = (final_value / config['backtest']['initial_capital'] - 1) * 100
    print(f"  {name}: ${final_value:,.2f} ({total_return:+.2f}%)")

In [ ]:
# Backtest AI strategy
print("Backtesting AI Portfolio...")
ai_results = backtester.backtest_strategy(allocations, returns, prices)

# Backtest benchmark (Buy & Hold SPY)
print("Backtesting Buy & Hold SPY...")
benchmark_results = backtester.create_benchmark_strategy(returns, prices, 'SPY')

# Backtest traditional 60/40
print("Backtesting Traditional 60/40...")
traditional_6040_results = backtester.create_traditional_6040(returns, prices)

print("\nAll backtests completed!")


## 7. Drawdown Analysis

In [ ]:
# Plot drawdowns
fig, ax = plt.subplots(figsize=(14, 7))

for name, results in strategies.items():
    portfolio_value = results['Portfolio_Value']
    cummax = portfolio_value.cummax()
    drawdown = (portfolio_value - cummax) / cummax
    
    ax.plot(drawdown.index, drawdown * 100, label=name, linewidth=2)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Drawdown (%)', fontsize=12)
ax.set_title('Portfolio Drawdown Over Time', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

print("\nMaximum Drawdowns:")
for name in strategies.keys():
    max_dd = comparison.loc[name, 'Max Drawdown']
    print(f"  {name}: {max_dd:.2%}")

## 8. Returns Analysis

In [ ]:
# Plot returns distributions
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for idx, (name, results) in enumerate(strategies.items()):
    returns_series = results['Returns']
    
    axes[idx].hist(returns_series * 100, bins=30, edgecolor='black', alpha=0.7, color=f'C{idx}')
    axes[idx].axvline(returns_series.mean() * 100, color='red', linestyle='--', 
                     linewidth=2, label=f'Mean: {returns_series.mean()*100:.2f}%')
    axes[idx].set_xlabel('Monthly Return (%)', fontsize=11)
    axes[idx].set_ylabel('Frequency', fontsize=11)
    axes[idx].set_title(f'{name}', fontsize=12, fontweight='bold')
    axes[idx].legend(fontsize=9)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Monthly returns comparison
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(len(ai_results))
width = 0.25

for idx, (name, results) in enumerate(strategies.items()):
    offset = (idx - 1) * width
    returns_pct = results['Returns'] * 100
    color = f'C{idx}'
    
    # Only plot every 6th bar to avoid clutter
    plot_indices = x[::6]
    plot_returns = returns_pct.values[::6]
    
    ax.bar(plot_indices + offset, plot_returns, width, label=name, alpha=0.8)

ax.set_xlabel('Period', fontsize=12)
ax.set_ylabel('Monthly Return (%)', fontsize=12)
ax.set_title('Monthly Returns Comparison (Every 6 Months)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Rolling metrics
window = 12  # 12-month rolling window

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Rolling returns
for name, results in strategies.items():
    rolling_returns = results['Returns'].rolling(window).mean() * 12 * 100
    axes[0].plot(rolling_returns.index, rolling_returns, label=name, linewidth=2)

axes[0].set_ylabel('Annualized Return (%)', fontsize=11)
axes[0].set_title(f'{window}-Month Rolling Return', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='black', linestyle='--', linewidth=0.8)

# Rolling volatility
for name, results in strategies.items():
    rolling_vol = results['Returns'].rolling(window).std() * np.sqrt(12) * 100
    axes[1].plot(rolling_vol.index, rolling_vol, label=name, linewidth=2)

axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_ylabel('Annualized Volatility (%)', fontsize=11)
axes[1].set_title(f'{window}-Month Rolling Volatility', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Risk-Adjusted Returns

In [ ]:
# Risk-return scatter plot
fig, ax = plt.subplots(figsize=(10, 7))

for name in strategies.keys():
    ret = comparison.loc[name, 'CAGR'] * 100
    vol = comparison.loc[name, 'Volatility'] * 100
    sharpe = comparison.loc[name, 'Sharpe Ratio']
    
    ax.scatter(vol, ret, s=200, label=name, alpha=0.7)
    ax.annotate(f'Sharpe: {sharpe:.3f}', 
               (vol, ret), 
               textcoords="offset points", 
               xytext=(0,10), 
               ha='center',
               fontsize=9)

ax.set_xlabel('Annualized Volatility (%)', fontsize=12)
ax.set_ylabel('CAGR (%)', fontsize=12)
ax.set_title('Risk-Return Profile', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Transaction Cost Analysis

In [ ]:
# Analyze transaction costs for AI portfolio
total_costs = ai_results['Transaction_Costs'].sum()
avg_monthly_cost = ai_results['Transaction_Costs'].mean()

print(f"Transaction Cost Analysis (AI Portfolio):")
print(f"  Total transaction costs: ${total_costs:,.2f}")
print(f"  Average monthly cost: ${avg_monthly_cost:,.2f}")
print(f"  As % of initial capital: {(total_costs/config['backtest']['initial_capital'])*100:.2f}%")

# Plot cumulative transaction costs
fig, ax = plt.subplots(figsize=(14, 6))
cumulative_costs = ai_results['Transaction_Costs'].cumsum()
ax.plot(cumulative_costs.index, cumulative_costs, linewidth=2, color='red')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Cumulative Transaction Costs ($)', fontsize=12)
ax.set_title('Cumulative Transaction Costs - AI Portfolio', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
plt.tight_layout()
plt.show()

## 11. Summary and Key Findings

### Performance Summary:

In [ ]:
# Create summary table
summary = pd.DataFrame({
    'Metric': [
        'Sharpe Ratio',
        'Sortino Ratio', 
        'CAGR',
        'Max Drawdown',
        'Volatility',
        'Calmar Ratio',
        'Final Value',
        'Total Return'
    ]
})

for name in strategies.keys():
    summary[name] = [
        f"{comparison.loc[name, 'Sharpe Ratio']:.4f}",
        f"{comparison.loc[name, 'Sortino Ratio']:.4f}",
        f"{comparison.loc[name, 'CAGR']*100:.2f}%",
        f"{comparison.loc[name, 'Max Drawdown']*100:.2f}%",
        f"{comparison.loc[name, 'Volatility']*100:.2f}%",
        f"{comparison.loc[name, 'Calmar Ratio']:.4f}",
        f"${comparison.loc[name, 'Final Value']:,.2f}",
        f"{comparison.loc[name, 'Total Return']*100:.2f}%"
    ]

print("="*80)
print("FINAL PERFORMANCE SUMMARY")
print("="*80)
display(summary)

### Key Insights:

1. **Sharpe Ratio**: The AI portfolio's risk-adjusted returns compared to benchmarks
2. **Dynamic Allocation**: AI adapts to changing market conditions
3. **Drawdown Protection**: Potential for better downside protection
4. **Transaction Costs**: Monthly rebalancing incurs costs but potentially improves returns

### Conclusion:

The AI-enhanced portfolio demonstrates the potential to improve upon traditional strategies by:
- Dynamically allocating across multiple asset classes
- Responding to economic indicators and market conditions
- Maintaining risk-adjusted returns while managing drawdowns